# Bidding

This is a tutorial for how to use the bidding engine

In [1]:
import os
os.chdir('..')

from nn.models import Models
from bots import BotBid
from bidding import bidding
from util import hand_to_str
from deck52 import random_deal
from sample import Sample
import conf
import numpy as np

np.set_printoptions(precision=2, suppress=True, linewidth=200)


In [2]:
models = Models.from_conf(conf.load('./config/default.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('./config/default.conf'))  # Load sampling strategies

Instructions for updating:
non-resource variables are not supported in the long term
INFO:tensorflow:Restoring parameters from ..\models/GIB/bidding_V2-3130000
INFO:tensorflow:Restoring parameters from ..\models/contract/contract-193200
INFO:tensorflow:Restoring parameters from ..\models/GIB/binfo_V2-3130000
INFO:tensorflow:Restoring parameters from ..\UCBC 2024/Models/lead_suit-999000
INFO:tensorflow:Restoring parameters from ..\UCBC 2024/Models/lead_nt-475000
INFO:tensorflow:Restoring parameters from ..\models/lr3_model/lr3-1000000
INFO:tensorflow:Restoring parameters from ..\models/single_dummy/single_dummy-32768000
INFO:tensorflow:Restoring parameters from ..\models/playing/lefty_nt-475000
INFO:tensorflow:Restoring parameters from ..\models/playing/dummy_nt-475000
INFO:tensorflow:Restoring parameters from ..\models/playing/righty_nt-475000
INFO:tensorflow:Restoring parameters from ..\models/playing/decl_nt-475000
INFO:tensorflow:Restoring parameters from ..\models/playing/lefty_sui

### Running through an example

In [3]:

vuln_ns, vuln_ew = False, False

# you sit West and hold:
hand = '6.AKJT82.762.K63'

# the auction goes:
auction = ['1D', '3S']
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, sampler, seat=2, dealer=0, verbose=False)

bid = bot_bid.bid(auction)
bid.to_dict()['candidates']

# what's your bid?

Loaded lib dds.dll


[{'call': '4H',
  'insta_score': 0.497,
  'expected_score': 284,
  'expected_tricks': 10.3,
  'adjustment': 25,
  'alert': 'False'},
 {'call': 'PASS',
  'insta_score': 0.498,
  'expected_score': 82,
  'expected_tricks': 9.0,
  'adjustment': 25,
  'alert': 'False'}]

In [4]:
auction = ['1D', '3S', '4H', '4S', '5H', 'PASS']
bid = bot_bid.bid(auction)
bid.to_dict()['candidates']

[{'call': 'PASS',
  'insta_score': 0.981,
  'expected_score': 160,
  'expected_tricks': 10.2,
  'adjustment': 0,
  'alert': 'False'}]

in the output above:
- `insta_score` reflects what the neural network prefers to bid
- `expected_score` is based on what you expect to score on difference samples (for example, 70% of the time you are making and 30% of the time you are going one off, then the expected score is 0.7 * 620 - 0.3 * 100)

### Samples consistent with the auction so far

Above we said that we are computig expected scores on samples consistent with the auction so far.

How do we get these samples?

We get some samples in the response we get from the bot (i.e `bid.samples`)

In [5]:
bid.samples

['A8x.Q9x.KQT9x.Jx KJT9xxx.xxx..T9x x.AKJT8x.xxx.Kxx Qx.x.AJ8xx.AQ8xx 0.69102',
 'x.Qx.AQJ98x.QJTx KQJ8xxx.9.x.A98x x.AKJT8x.xxx.Kxx AT9x.xxxx.KTx.xx 0.67535',
 'Ax.9xx.KT9x.A98x KT8xxxx.x.J8x.Qx x.AKJT8x.xxx.Kxx QJ9.Qxx.AQx.JTxx 0.67421',
 'x.xx.KQJT8x.AJTx KQT9xxx.x.A.Q8xx x.AKJT8x.xxx.Kxx AJ8x.Q9xx.9xx.9x 0.65984',
 'Tx.9x.KQJT98x.AJ AK98xxx.Qx..QTxx x.AKJT8x.xxx.Kxx QJx.xxx.Axx.98xx 0.64260',
 'A9x.xxx.KJxx.A8x KJT8xxx.xx.Tx.Tx x.AKJT8x.xxx.Kxx Qx.Q9.AQ98.QJ9xx 0.62197',
 'Axx.Qx.AK98x.98x KJ98xxx.9.Txx.Jx x.AKJT8x.xxx.Kxx QT.xxxx.QJ.AQTxx 0.61275',
 'AQx.xx.KQTxx.Q9x KJT8xxx.9xx.9.Jx x.AKJT8x.xxx.Kxx 9x.Qx.AJ8x.AT8xx 0.60011',
 'JT.xx.AKQ98xx.J8 KQ9xxxx.xx..AQxx x.AKJT8x.xxx.Kxx A8x.Q9x.JTx.T9xx 0.59392',
 'K9x.xx.KQ9x.AJxx QJT8xxx.Qx.JT.8x x.AKJT8x.xxx.Kxx Ax.9xx.A8xx.QT9x 0.59276',
 'Qx.Qxx.AKJTx.QT8 AT8xxxx.9x.9x.J9 x.AKJT8x.xxx.Kxx KJ9.xx.Q8x.Axxxx 0.59097',
 '9.Qxxx.KQJx.A98x AT8xxxx.x.A.QJTx x.AKJT8x.xxx.Kxx KQJx.9x.T98xx.xx 0.58950',
 'AQ.xx.KQ98xx.A8x K9xxxxx.Q.JTx.xx x.AK

each row is one sample. the hands are in the order: N E S W

(West always has the same cards because it's your hand and it's known to you)

In [6]:
# we got some samples above. if we want more, we can always get more

sample_hands, sorted_score, p_hcp, p_shp, good_quality = bot_bid.sample_hands_for_auction(['PAD_START','2N','PASS','3C','PASS','3D'], 0)
print("Good quality:", good_quality)
for i in range(sample_hands.shape[0]):
    print(f'{hand_to_str(sample_hands[i, 0, :])} {hand_to_str(sample_hands[i, 1, :])} {hand_to_str(sample_hands[i, 2, :])} {hand_to_str(sample_hands[i, 3, :])}')

Good quality: True
x.AKJT8x.xxx.Kxx K8x.Qxx.AKQ8.AQx QJxxx.xxx.Tx.9xx AT9x.9.J9xx.JT8x
x.AKJT8x.xxx.Kxx AK9.Qxx.AKJx.Axx QT8xx.9.8xx.98xx Jxxx.xxx.QT9.QJT
x.AKJT8x.xxx.Kxx AK.Qxx.AK9x.Axxx QJ8xxx.9x.Tx.JT8 T9xx.xx.QJ8x.Q9x
x.AKJT8x.xxx.Kxx Ax.Q9x.AKTx.AQJ8 QT98x..J98x.T9xx KJxxx.xxxx.Qx.xx
x.AKJT8x.xxx.Kxx KQx.Q9x.AKQ8x.AT JT8xx.x.xx.J98xx A9xx.xxx.JT9.Qxx
x.AKJT8x.xxx.Kxx AKQ.Qxx.AKQ98.Tx J98xx.x.Jxx.J98x Txxx.9xx.Tx.AQxx
x.AKJT8x.xxx.Kxx AQx.xxx.AKQJx.A9 KT98x.9.Tx.Jxxxx Jxxx.Qxx.98x.QT8
x.AKJT8x.xxx.Kxx AKx.Q9.AKT9.A9xx Q8xxx.xx.8x.JT8x JT9x.xxx.QJxx.Qx
x.AKJT8x.xxx.Kxx AQx.9x.AKQ98.AJx J8xxx.xx.x.QT98x KT9x.Qxx.JTxx.xx
x.AKJT8x.xxx.Kxx KQx.Qx.AKQ9x.AJx JT8xx.9xxx.8.Q8x A9xx.x.JTxx.T9xx
x.AKJT8x.xxx.Kxx Kxx.Qx.AKQx.AQTx QJT9x.9xxx.8.J8x A8xx.x.JT9xx.9xx
x.AKJT8x.xxx.Kxx AK8.Qx.AKJ.AT8xx Q9xxx.xx.Txx.J9x JTxx.9xx.Q98x.Qx
x.AKJT8x.xxx.Kxx AKQ.Qx.AQJ9.Q8xx T8xxx.x.K8xxx.JT J9xx.9xxx.T.A9xx
x.AKJT8x.xxx.Kxx AKx.Qx.KJ8.AQJ8x T9xxxx.x.ATx.9xx QJ8.9xxx.Q9xx.Tx
x.AKJT8x.xxx.Kxx AKx.Qx.AT9.A

How do we know on which sample 4H is making and on which sample it is not? It's the topic for another tutorial, but basically we could do either (1) solve the sample double dummpy or (2) estimate the outcome using a neural network

### Bidding through an entire auction for all four hands

Now we'll take a random deal and let the bot bid for each hand. We assume North deals and nobody is vulnerable.

In [7]:
hands = random_deal().split()
hands # N, E, S, W

['A87.AT52.Q.T8543',
 'QT9.J73.AT985.J7',
 '6532.84.KJ642.K9',
 'KJ4.KQ96.73.AQ62']

In [8]:
bidder_bots = [BotBid([False, False], hand, models, sampler,i, 0,False) for i, hand in enumerate(hands)]

In [9]:
auction = []  # since North deals, we don't need any 'PAD_START'

turn_i = 0  # whose turn is it to bid

while not bidding.auction_over(auction):
    auction.append(bidder_bots[turn_i].bid(auction).bid)
    turn_i = (turn_i + 1) % 4  # next player's turn
    
auction

expected_score 193.0


['PASS', 'PASS', 'PASS', '1N', 'PASS', 'PASS', 'PASS']